In [377]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
import pytorch_lightning as pl
import pandas as pd
import csv
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np
import optuna
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning import LightningModule
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import pytorch_lightning as pl
from sklearn.preprocessing import LabelEncoder


In [378]:
Data = pd.read_csv("/Users/zsl/Downloads/train.csv")

In [379]:
Data = Data.drop(['Id'],axis=1)

In [380]:
# Define the criteria for house categorization
def categorize_house(row):
    if row['HouseStyle'] == '1Story':
        if row['BldgType'] == '1Fam':
            if row['YearBuilt'] >= 2000:
                return 'Modern Single-Story'
            else:
                return 'Traditional Single-Story'
        else:
            return 'Other Single-Story'
    elif row['HouseStyle'] == '2Story':
        if row['BldgType'] == '1Fam':
            if row['YearRemodAdd'] >= 2010:
                return 'Newly Remodeled Two-Story'
            else:
                return 'Traditional Two-Story'
        else:
            return 'Other Two-Story'
    else:
        return 'Other Categories'

# Create the 'House Category' variable
Data['House Category'] = Data.apply(categorize_house, axis=1)

In [381]:
# Select 'SalePrice' and 'House Category' columns as target dataset
target_dataset = Data[['SalePrice', 'House Category']]
# Prepare your dataset
regression_targets = target_dataset['SalePrice']
# Dummify the categorical target variable
train_target_dummies = pd.get_dummies(target_dataset)
classification_targets = train_target_dummies
Data.drop('SalePrice', axis=1, inplace=True)
Data.drop('House Category', axis=1, inplace=True)
features = Data
Data = pd.concat([train_target_dummies, Data], axis=1)

In [382]:
features

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [383]:
Data 

,House Category_Modern Single-Story,House Category_Newly Remodeled Two-Story,House Category_Other Categories,House Category_Other Single-Story,House Category_Other Two-Story,House Category_Traditional Single-Story,House Category_Traditional Two-Story,MSSubClass,MSZoning,LotFrontage,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,0,0,0,0,0,1,60,RL,65.0,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,0,0,0,0,0,1,0,20,RL,80.0,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,0,0,0,0,0,0,1,60,RL,68.0,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,0,0,0,0,0,0,1,70,RL,60.0,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,0,0,0,0,0,0,1,60,RL,84.0,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,1,60,RL,62.0,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,0,0,0,0,0,1,0,20,RL,85.0,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,0,0,0,0,0,0,1,70,RL,66.0,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,0,0,0,0,0,1,0,20,RL,68.0,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [384]:
# Assuming train_data is your DataFrame containing the dataset

# Calculate the proportion of missing values for each column
missing_values_count = features.isnull().sum()
missing_values_proportion = features.isnull().mean()

# Print the count and proportion of missing values for each column if greater than 0
print("Missing values for each column:")
for column in features.columns:
    if missing_values_count[column] > 0:
        count = missing_values_count[column]
        proportion = missing_values_proportion[column]
        print(f"{column}: {count} missing values ({proportion:.2%})")

# Identify columns with missing values count more than 50%
columns_to_drop = missing_values_count[missing_values_proportion > 0.5].index

# Drop columns with missing values count more than 50%
train_data_dropped = features.drop(columns=columns_to_drop)

# Print the dropped columns
if len(columns_to_drop) > 0:
    print("Dropped columns:")
    for column in columns_to_drop:
        print(column)
else:
    print("No columns dropped.")

Missing values for each column:
LotFrontage: 259 missing values (17.74%)
Alley: 1369 missing values (93.77%)
MasVnrType: 8 missing values (0.55%)
MasVnrArea: 8 missing values (0.55%)
BsmtQual: 37 missing values (2.53%)
BsmtCond: 37 missing values (2.53%)
BsmtExposure: 38 missing values (2.60%)
BsmtFinType1: 37 missing values (2.53%)
BsmtFinType2: 38 missing values (2.60%)
Electrical: 1 missing values (0.07%)
FireplaceQu: 690 missing values (47.26%)
GarageType: 81 missing values (5.55%)
GarageYrBlt: 81 missing values (5.55%)
GarageFinish: 81 missing values (5.55%)
GarageQual: 81 missing values (5.55%)
GarageCond: 81 missing values (5.55%)
PoolQC: 1453 missing values (99.52%)
Fence: 1179 missing values (80.75%)
MiscFeature: 1406 missing values (96.30%)
Dropped columns:
Alley
PoolQC
Fence
MiscFeature


In [385]:
# Automatically select numerical and categorical columns
numerical_cols = features.select_dtypes(include=['int', 'float']).columns.tolist()
categorical_cols = features.select_dtypes(include=['object']).columns.tolist()

# Identify ordinal and high cardinality categorical columns
ordinal_cols = ['GarageCond', 'GarageQual', 'GarageFinish', 'FireplaceQu', 'Functional', 'KitchenQual', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC']
high_cardinality_cols = [col for col in categorical_cols if features[col].nunique() >= len(features) / 10]

# Numerical features preprocessing pipeline
numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical features preprocessing pipeline
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# High cardinality categorical features preprocessing pipeline
high_cardinality_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Ordinal categorical features preprocessing pipeline
ordinal_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

# Column Transformer to apply different transformations to different columns
preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('high_card', high_cardinality_transformer, high_cardinality_cols),
    ('ordinal', ordinal_transformer, ordinal_cols),
    ('cat', categorical_transformer, categorical_cols)
])

# Fit the preprocessor on your data
preprocessor.fit(features)

# Get the transformed column names
transformed_column_names = (
    numerical_cols
    + [f'{col}_encoded' for col in high_cardinality_cols]
    + ordinal_cols
    + preprocessor.transformers_[3][1].named_steps['encoder'].get_feature_names_out(categorical_cols).tolist()
)

# Apply the preprocessor to your data
Data_processed = preprocessor.transform(features)

# Create DataFrame with transformed data
Data_processed_df = pd.DataFrame(Data_processed, columns=transformed_column_names)
features = Data_processed_df
# Display the processed DataFrame
Data_processed_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.073375,-0.229372,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.872563,0.451936,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.073375,-0.093110,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.309859,-0.456474,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.073375,0.633618,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,-0.288653,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [404]:
# Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y_regression, y_classification):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y_regression = torch.tensor(y_regression.values, dtype=torch.float32)
        self.y_classification = torch.tensor(y_classification.values, dtype=torch.float32)  # Convert to float32

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y_regression[idx], self.y_classification[idx]

# Define your shared bottom model
class SharedBottomModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, activation_func):
        super(SharedBottomModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.activation_func = activation_func

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation_func(x)
        return x

# Define your multi-task model
class MyMultiTaskModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_classes, activation_func_regression, activation_func_classification):
        super(MyMultiTaskModel, self).__init__()
        self.shared_bottom_model = SharedBottomModel(input_dim, hidden_dim, activation_func_regression)
        self.fc_regression = nn.Linear(hidden_dim, 1)
        self.fc_classification = nn.Linear(hidden_dim, num_classes)
        self.activation_func_classification = activation_func_classification

    def forward(self, x):
        x = self.shared_bottom_model(x)
        regression_output = self.fc_regression(x)
        classification_output = self.fc_classification(x)
        return regression_output, classification_output

    def training_step(self, batch, batch_idx):
        x, y_regression, y_classification = batch
        regression_output, classification_output = self(x)
        
        # Compute the loss for regression task
        regression_loss = torch.sqrt(nn.MSELoss()(regression_output.flatten(), y_regression))
        
        # Compute the loss for classification task
        classification_loss = nn.CrossEntropyLoss()(classification_output, y_classification)
        
        # Total loss as a combination of both losses
        total_loss = regression_loss + classification_loss

        self.log('train_regression_loss', regression_loss)
        self.log('train_classification_loss', classification_loss)
        self.log('train_total_loss', total_loss)
        
        return total_loss

    def validation_step(self, batch, batch_idx):
        x, y_regression, y_classification = batch
        regression_output, classification_output = self(x)
        
        # Compute the RMSE loss for regression task
        regression_loss = torch.sqrt(nn.MSELoss()(regression_output.flatten(), y_regression))
        
        # Compute the Cross Enropy loss for classification task
        classification_loss = nn.CrossEntropyLoss()(classification_output, y_classification)
        
        # Total loss as a combination of both losses
        total_loss = regression_loss + classification_loss
        
        self.log('val_regression_loss', regression_loss)
        self.log('val_classification_loss', classification_loss)
        self.log('val_total_loss', total_loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.1)
        return optimizer

In [405]:
# Split your data into train and validation sets
X_train, X_val, y_regression_train, y_regression_val, y_classification_train, y_classification_val = train_test_split(
    features, regression_targets, classification_targets, test_size=0.2, random_state=42
)

# Create your custom datasets
train_dataset = CustomDataset(X_train, y_regression_train, y_classification_train)
val_dataset = CustomDataset(X_val, y_regression_val, y_classification_val)

# Create your data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Define the number of classes for the classification task
num_classes = len(set(classification_targets))

# Create an instance of your multi-task model
model = MyMultiTaskModel(input_dim=features.shape[1], hidden_dim=64, num_classes=num_classes, activation_func_regression=nn.ReLU(), activation_func_classification=nn.ReLU())

# Create a PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=10)

# Train your model
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                           | Type              | Params
---------------------------------------------------------------------
0 | shared_bottom_model            | SharedBottomModel | 19.4 K
1 | fc_regression                  | Linear            | 65    
2 | fc_classification              | Linear            | 455   
3 | activation_func_classification | ReLU              | 0     
---------------------------------------------------------------------
19.9 K    Trainable params
0         Non-trainable params
19.9 K    Total params
0.080     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [406]:
# After trainer.fit()
val_loss = trainer.callback_metrics['val_total_loss']
print(f"Validation Loss: {val_loss:.4f}")

Validation Loss: 29130.3184


In [407]:
activation_functions = [
    nn.ReLU(),
    nn.Sigmoid(),
    nn.Tanh(),
    nn.LeakyReLU()
]

optimizers = [
    optim.Adam,
    optim.SGD,
    optim.RMSprop,
    optim.Adagrad
]

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Activation Function', 'Optimizer', 'Validation Loss'])

# Loop over the activation functions and optimizers
for activation_func in activation_functions:
    for optimizer_class in optimizers:
        # Create an instance of the optimizer
        optimizer = optimizer_class(model.parameters(), lr=0.1)
        
        # Set the activation function in the shared bottom model
        model.shared_bottom_model.activation_func = activation_func
        
        # Create a PyTorch Lightning Trainer
        trainer = pl.Trainer(max_epochs=10)
        
        # Train your model
        trainer.fit(model, train_loader, val_loader)
        
        # Get the validation loss
        val_loss = trainer.logged_metrics['val_total_loss']
        
        # Store the results in the DataFrame
        results_df = results_df.append({'Activation Function': str(activation_func),
                                        'Optimizer': optimizer_class.__name__,
                                        'Validation Loss': val_loss.item()}, ignore_index=True)

# Print the results
print(results_df)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                           | Type              | Params
---------------------------------------------------------------------
0 | shared_bottom_model            | SharedBottomModel | 19.4 K
1 | fc_regression                  | Linear            | 65    
2 | fc_classification              | Linear            | 455   
3 | activation_func_classification | ReLU              | 0     
---------------------------------------------------------------------
19.9 K    Trainable params
0         Non-trainable params
19.9 K    Total params
0.080     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (37) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a low

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


               Activation Function Optimizer  Validation Loss
0                           ReLU()      Adam     26911.177734
1                           ReLU()       SGD     26489.443359
2                           ReLU()   RMSprop     26743.771484
3                           ReLU()   Adagrad     26130.900391
4                        Sigmoid()      Adam    196263.218750
5                        Sigmoid()       SGD    194590.000000
6                        Sigmoid()   RMSprop    192879.984375
7                        Sigmoid()   Adagrad    191151.328125
8                           Tanh()      Adam    189003.828125
9                           Tanh()       SGD    186855.031250
10                          Tanh()   RMSprop    184711.406250
11                          Tanh()   Adagrad    182574.984375
12  LeakyReLU(negative_slope=0.01)      Adam     38846.250000
13  LeakyReLU(negative_slope=0.01)       SGD     27869.996094
14  LeakyReLU(negative_slope=0.01)   RMSprop     25719.996094
15  Leak

/var/folders/y8/z84_qvjd3838gv1wm23qyc4h0000gn/T/ipykernel_38717/675189805.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Activation Function': str(activation_func),


In [408]:
results_df = results_df.sort_values('Validation Loss')
results_df

,Activation Function,Optimizer,Validation Loss
14,LeakyReLU(negative_slope=0.01),RMSprop,25719.996094
3,ReLU(),Adagrad,26130.900391
15,LeakyReLU(negative_slope=0.01),Adagrad,26207.580078
1,ReLU(),SGD,26489.443359
2,ReLU(),RMSprop,26743.771484
0,ReLU(),Adam,26911.177734
13,LeakyReLU(negative_slope=0.01),SGD,27869.996094
12,LeakyReLU(negative_slope=0.01),Adam,38846.250000
11,Tanh(),Adagrad,182574.984375
10,Tanh(),RMSprop,184711.406250


In [415]:
def objective(trial):
    # Sample hyperparameters to tune
    hidden_dim = trial.suggest_int("hidden_dim", 32, 128)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)

    # Create an instance of your multi-task model with the sampled hyperparameters
    model = MyMultiTaskModel(input_dim=features.shape[1], hidden_dim=hidden_dim, num_classes=num_classes, activation_func_regression=nn.LeakyReLU(), activation_func_classification=nn.LeakyReLU())

    # Define the optimizer with the sampled learning rate
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Create a PyTorch Lightning Trainer with the desired configuration
    trainer = pl.Trainer(max_epochs=10)

    # Train your model
    trainer.fit(model, train_loader, val_loader)

    # Evaluate the model on the validation set
    val_loss = trainer.logged_metrics['val_total_loss']
    
    return val_loss

In [416]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# Print the best hyperparameters and objective value
best_params = study.best_params
best_value = study.best_value
print("Best parameters:", best_params)
print("Best value:", best_value)

[I 2023-06-11 00:44:09,372] A new study created in memory with name: no-name-018a1e19-890a-4f24-a27d-6e4baa2c549c
/var/folders/y8/z84_qvjd3838gv1wm23qyc4h0000gn/T/ipykernel_38717/2278947719.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                           | Type              | Params
---------------------------------------------------------------------
0 | shared_bottom_model            | SharedBottomModel | 13.6 K
1 | fc_regression                  | Linear            | 46    
2 | fc_classification              | Linear            | 322   
3 | activation_func_classification | Lea

Sanity Checking: 0it [00:00, ?it/s]

/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (37) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a low

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
[I 2023-06-11 00:45:49,971] Trial 9 finished with value: 29883.978515625 and parameters: {'hidden_dim': 56, 'learning_rate': 0.002680359045443441}. Best is trial 2 with value: 28720.583984375.


Best parameters: {'hidden_dim': 89, 'learning_rate': 0.0002682288030127745}
Best value: 28720.583984375


In [426]:
# Get the best hyperparameters
best_params = study.best_params

# Extract the best hyperparameters
hidden_dim = best_params['hidden_dim']
learning_rate = best_params['learning_rate']

for activation_func in activation_functions:
    for optimizer_class in optimizers:
        # Create an instance of the optimizer
        optimizer = optimizer_class(model.parameters(), lr=0.1)
        
        # Set the activation function in the shared bottom model
        model.shared_bottom_model.activation_func = activation_func
        
        # Create a PyTorch Lightning Trainer
        trainer = pl.Trainer(max_epochs=10)
        
        # Train your model
        trainer.fit(model, train_loader, val_loader)
        
        # Get the validation loss
        val_loss = trainer.logged_metrics['val_total_loss']
        
        # Check if this combination of parameters is the best so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_activation_func = activation_func
            best_optimizer = optimizer_class.__name__


# Create an instance of your multi-task model with the best hyperparameters
model = MultiTaskModel(input_dim=features.shape[1], hidden_dim=hidden_dim,
                       num_classes=num_classes,
                       activation_func_regression=activation_func,
                       activation_func_classification=activation_func)

# Create an optimizer with the best learning rate
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create a PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=10)

# Train your model
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                           | Type              | Params
---------------------------------------------------------------------
0 | shared_bottom_model            | SharedBottomModel | 19.4 K
1 | fc_regression                  | Linear            | 65    
2 | fc_classification              | Linear            | 455   
3 | activation_func_classification | ReLU              | 0     
---------------------------------------------------------------------
19.9 K    Trainable params
0         Non-trainable params
19.9 K    Total params
0.080     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [427]:
best_val_loss

tensor(27436.1602)